<h1 align="center"><font color='green'>Project Practice. Elina Kantseva. August 2024</font></h1>

<a class="anchor" id="toc"></a>
### <font color='289C4E'>Table of Contents<font><a class='anchor' id='top'></a>
- [__Part 1. Decay Constants: calculationg and comparesment__](#0)
- [Introduction](#1)
- [Theoretical Basis](#2)
- [Evaluation: Dataset 1](#3)
    - [Reading in the Data](#3.1)
    - [Full and sliced Data](#3.2)
    - [Implementing the filter](#3.3)
    - [Smoothing the data](#3.4)
    - [Comparesment of data smoothing algorithms](#3.5)
    - [Implementing of data smoothing algorithms](#3.6)
    - [Fitting](#3.7)
    - [Inverting the Function and Fitting again](#3.8)
    
__before evaluating Datasets 2,3,4 and the Part 2 of this Notebook the Dataset 1 section of this notebook should be run, because it defines important functions__ 
    
- [Evaluation: Dataset 2](#4)
- [Evaluation: Dataset 3](#5)
- [Evaluation: Dataset 4](#6)
- [Results for Decay Constants $K$](#7)
- [__Part 2. Friction velocity: qualitative calculationg__ ](#8)
    - [Introduction](#8.1)
    - [Theoretical Basis](#8.2)
    - [Evaluation: Dataset 1](#8.3)
    - [Results for the friction velocity $u*_w$](#8.4)

<a class="anchor" id="0"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='black'>Part 1. Decay Constants: calculationg and comparesment</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

<a class="anchor" id="1"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Introduction</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

This Jupyter Notebook is based on the code by Kerstin Krall and the Bachelor's thesis (2011) by Maximilian Bopp. The measurements were conducted at different times in the wind-wave channel "Aeolotron" in Heidelberg.  __<font color='green'>The goal</font>__ of the internship was to obtain the decay coefficients \(K\) from various datasets (without surface film, with soluble and insoluble films) and to compare them with the values from M. Bopp's work.

<a class="anchor" id="2"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Theoretical Basis</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

As mentioned in the work of M. Bopp, the water behavior in the channel is described by the force balance:

\begin{align}
\frac{dp}{dt}= F_{wind} - \sum F_{wände} - F_{reibung}
\end{align}

The left side of the equation can be written as the following expression:

\begin{align}
\frac{dp}{dt} = m \frac{du}{dt} = V \rho \frac{du}{dt}
\end{align}

As for the right side, for a general force acting on the water body with the surface area \( A \), the following applies:

\begin{align}
F = A \tau_{xz} = A \rho {u_{\ast}}^2
\end{align}

It's true that $u_{\ast} = C u$ (C = a constant), therefor it follows:

\begin{align}
F = A \tau_{xz} = A \rho {u_{\ast}}^2 = A \rho C^2 u^2
\end{align}

For a frictional force exerted by the immersed body on the water, the following applies:

\begin{align}
F_R = \frac{1}{2}c_w \rho F u^2
\end{align}

Here, $ F $ is the cross-sectional area, and $ c_w $ is the drag coefficient.

After the wind is turned off, $ F_{wind} $ disappears. Thus, the force balance can be simplified as follows:

\begin{align}
V \rho \frac{du}{dt} =  - \sum A \rho {C^2} u^2 - \frac{1}{2}c_w \rho F u^2 = - K u^2
\end{align}

It follows that:

\begin{align}
\int_{u(0)}^{u(t)} \frac{du'}{u'}^2 = - \int_0^t K dt'
\end{align}

\begin{align}
\rightarrow \frac{1}{u(t)} = \frac{1}{u(0)} + Kt
\end{align}

\begin{align}
\rightarrow {u(t)} = \frac{u(0)}{1 + u(0)Kt}
\end{align}

In [1]:
import numpy as np
#import cv2
import matplotlib.pyplot as plt
import os
from scipy.interpolate import CubicSpline
import matplotlib
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import datetime as datetime
from datetime import datetime as dt

In [2]:
params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)


# ein paar Einstellungen zur Schriftart:
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 12}
#updaten der matplotlib 
matplotlib.rc('font', **font)

ZFZF = lambda x,pos: str(round(x,7)).rstrip('0').rstrip('.')

#%% colors
indigo='#332288'
cyan='#33bbee'
teal='#22aa88'
green='#117733'
olive='#999933'
sand='#eedd77'
rose='#DD6677'
wine='#882255'
purple='#aa4499'
blue='#0077bb'
orange='#ee7733'
red='#cc3311'
magenta='#ee3377'
gray='#AAAAAA'

teal2='#009988'

#%%

In [3]:
#%%
def concatenate_files(file1_path, file2_path, output_file_path):
    # Read the contents of the first file
    with open(file1_path, 'r') as file1:
        content1 = file1.read()

    # Read the contents of the second file
    with open(file2_path, 'r') as file2:
        content2 = file2.read()

    # Concatenate the contents
    combined_content = content1 + content2

    # Write the combined content to the output file
    with open(output_file_path, 'w') as output_file:
        output_file.write(combined_content)
        
        
def replace_commas_with_space_in_place(file_path):
    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Replace commas with spaces
    modified_content = content.replace(',', ' ')

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.write(modified_content)
#%%   

def find_position(listt, bound):
    c = -1
    for count, value in enumerate(listt):
        if value >= bound:
            c = count
            break
    return c

def filter(listX, listY):
    last = listY[0]
    for i, j in enumerate(listY):
        if (j > last*1.7):
            listY.pop(i)
            listX.pop(i)
        try: last = listY[i]
        except: last = listY[i-1]
    return (listX, listY)


<a class="anchor" id="3"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Evaluation: Dataset 1</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

Data: Water Velocity [m/s] against time of the day [h] In following we look into the data taken at 15:50 with no film.

As we are only interested in the data after x $\approx$ 16.0 h (after the wind was switched of) we need to slice the data. We also need to filter the data, to minimize the noise. It seems reasonable, to first slice and then filter, so that we need to filter less.I already wrote and tested my filter algorithm on all the data files (so i intend to keep it) that works with lists. My initial slicing technique worked with np.arrays, as I wanted to work with hours for X, not with seconds. Now i decided to stick to the lists, the time will be in the seconds then.

<a class="anchor" id="3.1"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Reading in the Data</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [4]:
#

%matplotlib widget

def readthefile(filename):
    environmentfile = filename
    #"data/2024_07_31_10_47_54_751214-2-anemometer-microseconds-spindown-ab-15.05.asc"
    #"data/2024_07_25_11_01_13_491761-2-anemometer-microseconds-spindown-ab-16.00.asc"

    plotpath=r"data"


    print(environmentfile)


    a1list=[]
    a2list=[]
    a1elapsedlist=[]
    a2elapsedlist=[]

    with open(environmentfile,'r') as infile:
        lines=infile.readlines()
        data=lines[1].split()
        datestring=data[0]+" "+data[1]
        starttime=dt.strptime(datestring,"%Y-%m-%d %H:%M:%S.%f")
        datestring=data[0]
        startday=dt.strptime(datestring,"%Y-%m-%d")
        a1last=2.0
        a2last=2.0
        for line in lines[1:]:
            data=line.split()
            datestring=data[0]+" "+data[1]
            try:
                elapsedtime=dt.strptime(datestring,"%Y-%m-%d %H:%M:%S.%f")-startday
            except ValueError:
                elapsedtime=dt.strptime(datestring,"%Y-%m-%d %H:%M:%S")-startday
            
            elapsedtime=elapsedtime.total_seconds()
        
            if(data[2]=="a1:"):  
                vel=1000000.0/float(data[4])/72
                if ( (vel<3*a1last and vel>0.25) or (vel>0 and vel<0.5)):
                    #print(vel)
                    a1elapsedlist.append(elapsedtime) #x
                    a1list.append(vel) #y
                    a1last=vel
            if(data[2]=="a2:"):
                vel=1000000.0/float(data[4])/72
                if ((vel<3*a2last and vel>0.25) or (vel>0 and vel<0.5)):
                    a2elapsedlist.append(elapsedtime)
                    a2list.append(vel)
                    a2last=vel
        
    a1list.pop(0)
    a2list.pop(0)
    a1elapsedlist.pop(0)
    a2elapsedlist.pop(0)
    
    return (a1elapsedlist, a1list, a2elapsedlist, a2list)

a1elapsedlist,a1list, a2elapsedlist,a2list = readthefile("data/2024_07_26_10_38_10_194617-2-anemometer-microseconds-spindown-ab-15.50.asc")

data/2024_07_26_10_38_10_194617-2-anemometer-microseconds-spindown-ab-15.50.asc


<a class="anchor" id="3.2"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Full and sliced Data</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>


In [5]:
def plotfullandsliced(a1elapsedlist,a1list,a2elapsedlist,a2list, bound):
    
    fig, ax = plt.subplots(figsize = (9,6))
    #plt.ylim([-0.01,0.2])
    #plt.xlim([16.0,16.9])

    plt.xlabel('time of day [s]')
    plt.ylabel('water velocity [m/s]')
    plt.title("Full Data")


    ax.plot(a1elapsedlist,a1list,label="water anemometer a1")
    ax.plot(a2elapsedlist,a2list,label="water anemometer a2")

    plt.grid(visible=True, which='major', color='grey', linestyle='-')
    ax.legend()


    plt.show()

    #cut everything that not out of the range "after the wind stopped" off
    ind1 = find_position(a1elapsedlist, bound*3600) #15.0 #16.0
    print(ind1)
    ind2 = find_position(a2elapsedlist, bound*3600) #15.0 #16.0
    print(ind2)
    a1elapsed_s = a1elapsedlist[ind1::]
    a2elapsed_s = a2elapsedlist[ind2::]
    a1_s = a1list[ind1::]
    a2_s = a2list[ind2::]

    fig, ax = plt.subplots(figsize = (9,6))
    #plt.ylim([-0.01,0.2])
    #plt.xlim([16.0,16.9])

    plt.xlabel('time of day [s]')
    plt.ylabel('water velocity [m/s]')
    plt.title("Sliced Data")


    ax.plot(a1elapsed_s,a1_s,label="water anemometer a1")
    ax.plot(a2elapsed_s,a2_s,label="water anemometer a2")

    plt.grid(visible=True, which='major', color='grey', linestyle='-')
    ax.legend()


    plt.show()
    
    return (a1elapsed_s, a1_s, a2elapsed_s, a2_s)

a1elapsed_s, a1_s, a2elapsed_s, a2_s = plotfullandsliced(a1elapsedlist,a1list,a2elapsedlist,a2list, 16.0)

/usr/lib/python3/dist-packages/matplotlib/__init__.py:882: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

91485
99383


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<a class="anchor" id="3.3"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Implementing the filter</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [6]:
def plotfilter(a1elapsed_s, a1_s, a2elapsed_s, a2_s):

    #filter
    a1elapsed_fs = filter(a1elapsed_s, a1_s)[0]
    a1_fs = filter(a1elapsed_s, a1_s)[1]

    a2elapsed_fs = filter(a2elapsed_s, a2_s)[0]
    a2_fs = filter(a2elapsed_s, a2_s)[1]

    fig, ax = plt.subplots(figsize = (9,6))
    #plt.ylim([-0.01,0.2])
    #plt.xlim([16.0,16.9])

    plt.xlabel('time of day [s]')
    plt.ylabel('water velocity [m/s]')
    plt.title("Sliced Filtered Data")


    ax.plot(a1elapsed_fs,a1_fs,label="water anemometer a1")
    ax.plot(a2elapsed_fs,a2_fs,label="water anemometer a2")

    plt.grid(visible=True, which='major', color='grey', linestyle='-')
    ax.legend()


    plt.show()
    
    return a1elapsed_fs, a1_fs, a2elapsed_fs, a2_fs

a1elapsed_fs, a1_fs, a2elapsed_fs, a2_fs = plotfilter(a1elapsed_s, a1_s, a2elapsed_s, a2_s)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As the both anemometers measured the same velocity, __we will now stick to the one, that messured more - water anemometer a2__.

<a class="anchor" id="3.4"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Smoothing the data</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

After some attempts in implementing curve_fit function on the filtered data, it's been established that even the filtered data has too much noise. After conducting some reaserch in the Internet the technique of the data smoothing comes into question. According to the Internet the following information about each method should help with choosing the right technique:

- Moving Average: Best for reducing random noise, simple to implement.

- Exponential Smoothing: Good for data with trends and seasonal patterns.

- Savitzky-Golay Filter: Preserves the original shape and features of the signal better than moving averages.

- Gaussian Smoothing: Good for general smoothing with a specific smoothing width.

- Median Filter: Effective for removing spikes and outliers.

- Lowess: Ideal for smoothing data with complex, non-linear trends.

In the end I picked the Lowess method, because it provided the best smoothing result for my data. The next cell is a side-by-side comparesment of many different data smoothing algorithms on randomly generated noisy data.

<a class="anchor" id="3.5"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Comparesment of data smoothing algorithms</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [7]:
#side-by-side comparesment of many different data smoothing algorithms
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter
from scipy.signal import medfilt
import statsmodels.api as sm

def simple_moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

def weighted_moving_average(data, weights):
    return np.convolve(data, weights, mode='valid')

def simple_exponential_smoothing(data, alpha):
    return pd.Series(data).ewm(alpha=alpha).mean().values

def double_exponential_smoothing(data, seasonal_periods=None):
    model = ExponentialSmoothing(data, trend='add', seasonal_periods=seasonal_periods).fit()
    return model.fittedvalues

def savitzky_golay_smoothing(data, window_length, polyorder):
    return savgol_filter(data, window_length, polyorder)

def gaussian_smoothing(data, sigma):
    return gaussian_filter(data, sigma=sigma)

def median_filter_smoothing(data, kernel_size):
    return medfilt(data, kernel_size=kernel_size)

def lowess_smoothing(data, frac):
    x = np.arange(len(data))
    smoothed = sm.nonparametric.lowess(data, x, frac=frac)
    return smoothed[:, 1]

# Generate synthetic noisy data
np.random.seed(0)
x = np.linspace(0, 10, 100)
y = np.sin(x) + 0.5 * np.random.normal(size=x.size)

# Apply smoothing techniques
y_sma = simple_moving_average(y, window_size=5)
y_wma = weighted_moving_average(y, np.array([0.1, 0.2, 0.4, 0.2, 0.1]))
y_exp = simple_exponential_smoothing(y, alpha=0.3)
y_sg = savitzky_golay_smoothing(y, window_length=11, polyorder=2)
y_gauss = gaussian_smoothing(y, sigma=2)
y_med = median_filter_smoothing(y, kernel_size=5)
y_lowess = lowess_smoothing(y, frac=0.2)

# Plot the original and smoothed data
plt.figure(figsize=(9, 6))
plt.plot(x, y, 'k-', label='Original Data')
plt.plot(x[2:-2], y_sma, 'b-', label='Simple Moving Average')
plt.plot(x[2:-2], y_wma, 'c-', label='Weighted Moving Average')
plt.plot(x, y_exp, 'g-', label='Exponential Smoothing')
plt.plot(x, y_sg, 'm-', label='Savitzky-Golay Filter')
plt.plot(x, y_gauss, 'y-', label='Gaussian Smoothing')
plt.plot(x, y_med, 'r-', label='Median Filter')
plt.plot(x, y_lowess, 'orange', label='Lowess Smoothing')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Comparison of Smoothing Techniques')
plt.show()


/usr/lib/python3/dist-packages/matplotlib/__init__.py:882: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<a class="anchor" id="3.6"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Implementing of data smoothing algorithms</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [8]:
#smoothing

a2_sma = simple_moving_average(a2_fs, window_size=101)

a2_exp = simple_exponential_smoothing(a2_fs, alpha=0.1)

a2_sg = savitzky_golay_smoothing(a2_fs, window_length=101, polyorder=2)

a2_gauss = gaussian_smoothing(a2_fs, sigma=100)

a2_med = median_filter_smoothing(a2_fs, kernel_size=101)

a2_lowess = lowess_smoothing(a2_fs, frac=0.2)

labels1 = ['data a2', 'Savitzky-Golay', 'Lowess smoothing']
a2_data1 = [a2_fs, a2_sg, a2_lowess]

labels2 = ['data a2', 'median filter', 'Gaussian smoothing']
a2_data2 = [a2_fs, a2_med, a2_gauss]

#%%

fig1, ax1 = plt.subplots(figsize = (9,6))

plt.xlabel('time of day [s]')
plt.ylabel('water velocity [m/s]')
plt.title("Smoothing 1")

   
for i in range(len(a2_data1)):
    ax1.plot(a2elapsed_fs, a2_data1[i], label = labels1[i])

plt.grid(visible=True, which='major', color='grey', linestyle='-')

ax1.legend()
plt.show()


#%%

fig2, ax2 = plt.subplots(figsize = (9,6))

plt.xlabel('time of day [s]')
plt.ylabel('water velocity [m/s]')
plt.title("Smoothing 2")

for i in range(len(a2_data2)):
    ax2.plot(a2elapsed_fs, a2_data2[i], label = labels2[i])
    
plt.grid(visible=True, which='major', color='grey', linestyle='-')

ax2.legend()
plt.show()
    
#%%

fig3, ax3 = plt.subplots(figsize = (9,6))

plt.xlabel('time of day [s]')
plt.ylabel('water velocity [m/s]')
plt.title("Smoothing 3")
    
ax3.plot(a2elapsed_fs, a2_fs, label = 'data')

ax3.plot(a2elapsed_fs,  a2_exp, label = 'exponential smoothig')

ax3.plot(a2elapsed_fs[50:-50], a2_sma, label = 'moving average')

plt.grid(visible=True, which='major', color='grey', linestyle='-')

ax3.legend()
plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<a class="anchor" id="3.7"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Fitting</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [9]:
def koef_func(t, K, u0):
        return (u0/(1 + K*u0*t))

def fittingandsmoothing(a2_fs, a2elapsed_fs, a2_lowess, bound_up, bound_down):

    #smoothing
    a2_lowess = lowess_smoothing(a2_fs, frac=0.2)

    #para = [0.01, 0.9]
    popt, pcov = curve_fit(koef_func, np.array(a2elapsed_fs)-(bound_down*3600), a2_lowess)
    xfine = np.linspace(0, bound_up*3600 - bound_down*3600) 

    fig, ax = plt.subplots(figsize = (9,6))

    plt.xlabel('time of day [s]')
    plt.ylabel('water velocity [m/s]')
    plt.title("Smoothing and Fitting")

    ax.plot(a2elapsed_fs, a2_fs, label = "data a2")
    ax.plot(xfine+(bound_down*3600), koef_func(xfine, *popt), label = 'fit Lowess')
    ax.plot(a2elapsed_fs, a2_lowess, label = "Lowess smoothing")

    plt.grid(visible=True, which='major', color='grey', linestyle='-')

    ax.legend()
    plt.show()
    
    return (popt[0], pcov[0][0]**0.5)
    
K = fittingandsmoothing(a2_fs, a2elapsed_fs, a2_lowess, 16.9, 16.0)
print('K=', K[0], '+/-', K[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.0220030254566865 +/- 6.318247473260458e-05


I tried this fitting algorithm with the data smoothed via Savitzky-Golay and got the exact same fit. After I was done, Kerstin suggested I invert the fitting function, so that it becomes linear. This method is supposed to be adventageous as more accurate.

<a class="anchor" id="3.8"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h2 style="margin: 0; text-align: center; width: 100%">
        <font color='gray'>Inverting the Function and Fitting again</font>
    </h2>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [10]:
def koef_func_inverse(t, K, C):
    return (C + K*t)

def inversefitting(a2_fs, a2elapsed_fs, a2_lowess, bound_up, bound_down):

    #para = [0.01, 0.9]
    popt, pcov = curve_fit(koef_func_inverse, np.array(a2elapsed_fs)-(bound_down*3600), 1/a2_lowess)
    xfine = np.linspace(0, bound_up*3600-bound_down*3600) 

    fig, ax = plt.subplots(figsize = (9,6))

    plt.xlabel('time of day [s]')
    plt.ylabel('inverse water velocity [s/m]')
    plt.title("Inverted Fit")

    ax.plot(a2elapsed_fs, 1/np.array(a2_fs), label = "data a2")
    ax.plot(xfine+(bound_down*3600), koef_func_inverse(xfine, *popt), label = 'fit')
    ax.plot(a2elapsed_fs, 1/np.array(a2_lowess), label = "lowess smoothing")

    plt.grid(visible=True, which='major', color='grey', linestyle='-')

    ax.legend()
    plt.show()
    
    return (popt[0], pcov[0][0]**0.5)

#%%
K = inversefitting(a2_fs, a2elapsed_fs, a2_lowess, 16.9, 16.0)
print('K=', K[0], '+/-', K[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.036550479260923585 +/- 0.0001648838203410883


After observing the data exploading for $t > \approx 59000 \, s$ I was recomended to only use the data below $58500$ this point.

In [11]:
ind = find_position(a2elapsed_fs, 58500) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 58500/3600, 16.0)
print('K=', K[0], '+/-', K[1])

3495


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.021157615372626275 +/- 3.368009771215271e-05


Next i am going to try and use only the very beggining of the plot to fit my function and get my $K$

In [12]:
ind = find_position(a2elapsed_fs, 58000) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 58000/3600, 16.0)
print('K=', K[0], '+/-', K[1])


1981


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.017714232885995263 +/- 2.3113446951216764e-05


<a class="anchor" id="4"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Evaluation: Dataset 2</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

__before evaluating Datasets 2,3 and 4 the Dataset 1 section of this notebook should be run, because it defines important functions.__ 

In [13]:
#reading in the data

a1elapsedlist,a1list, a2elapsedlist,a2list = readthefile("data/2024_07_31_10_47_54_751214-2-anemometer-microseconds-spindown-ab-15.05.asc")

#printing full and sliced

a1elapsed_s, a1_s, a2elapsed_s, a2_s = plotfullandsliced(a1elapsedlist,a1list,a2elapsedlist,a2list, 15.0)

#implementing the filter

a1elapsed_fs, a1_fs, a2elapsed_fs, a2_fs = plotfilter(a1elapsed_s, a1_s, a2elapsed_s, a2_s)

#smoothing and fitting
a2_lowess = lowess_smoothing(a2_fs, frac=0.2)

K = fittingandsmoothing(a2_fs, a2elapsed_fs, a2_lowess, 15.9, 15.0)
print('K=', K[0], '+/-', K[1])

#inverted fit

K = inversefitting(a2_fs, a2elapsed_fs, a2_lowess, 15.9, 15.0)
print('K=', K[0], '+/-', K[1])

#zoomed in fit

ind = find_position(a2elapsed_fs, 55000) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 55000/3600, 15.0)
print('K=', K[0], '+/-', K[1])


#closer zoomed in fit

ind = find_position(a2elapsed_fs, 54500) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 54500/3600, 15.0)
print('K=', K[0], '+/-', K[1])

data/2024_07_31_10_47_54_751214-2-anemometer-microseconds-spindown-ab-15.05.asc


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

7613
15646


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

K= 0.040847785237380196 +/- 0.0003112847490102252
2340


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.020101318229077236 +/- 0.0001558216238478331
1358


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.009797766410070019 +/- 0.00022762001988920643


<a class="anchor" id="5"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Evaluation: Dataset 3</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [14]:
#reading in the data

a1elapsedlist,a1list, a2elapsedlist,a2list = readthefile("data/2024_07_25_11_01_13_491761-2-anemometer-microseconds-spindown-ab-16.00.asc" )

#printing full and sliced

a1elapsed_s, a1_s, a2elapsed_s, a2_s = plotfullandsliced(a1elapsedlist,a1list,a2elapsedlist,a2list, 16.0)

#implementing the filter

a1elapsed_fs, a1_fs, a2elapsed_fs, a2_fs = plotfilter(a1elapsed_s, a1_s, a2elapsed_s, a2_s)

#smoothing and fitting
a2_lowess = lowess_smoothing(a2_fs, frac=0.2)

K = fittingandsmoothing(a2_fs, a2elapsed_fs, a2_lowess, 16.9, 16.0)
print('K=', K[0], '+/-', K[1])

#inverted fit

K = inversefitting(a2_fs, a2elapsed_fs, a2_lowess, 16.9, 16.0)
print('K=', K[0], '+/-', K[1])

#zoomed in inverted fit

ind = find_position(a2elapsed_fs, 60000) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 60000/3600, 16.0)
print('K=', K[0], '+/-', K[1])

#closer inverted fit

ind = find_position(a2elapsed_fs, 59000) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 59000/3600, 16.0)
print('K=', K[0], '+/-', K[1])


data/2024_07_25_11_01_13_491761-2-anemometer-microseconds-spindown-ab-16.00.asc


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

67334
76771


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/lib/python3/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


...

K= 0.02813872586034627 +/- 0.00010428664757751137
9484


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.023719722847794476 +/- 6.977665764970504e-05
7954


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.016536569499209786 +/- 2.2482911624194664e-05


<a class="anchor" id="6"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Evaluation: Dataset 4</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In [15]:
#reading in the data

a1elapsedlist,a1list, a2elapsedlist,a2list = readthefile("data/2024_08_07_10_55_58_657286-2-anemometer-microseconds.asc")

#printing full and sliced

a1elapsed_s, a1_s, a2elapsed_s, a2_s = plotfullandsliced(a1elapsedlist,a1list,a2elapsedlist,a2list, 16.3)

#implementing the filter

a1elapsed_fs, a1_fs, a2elapsed_fs, a2_fs = plotfilter(a1elapsed_s, a1_s, a2elapsed_s, a2_s)

#smoothing and fitting
a2_lowess = lowess_smoothing(a2_fs, frac=0.2)

K = fittingandsmoothing(a2_fs, a2elapsed_fs, a2_lowess, 17.0, 16.3)
print('K=', K[0], '+/-', K[1])

#inverted fit

K = inversefitting(a2_fs, a2elapsed_fs, a2_lowess, 17.0, 16.3)
print('K=', K[0], '+/-', K[1])

#zoomed in inverted fit

ind = find_position(a2elapsed_fs, 60000) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 60000/3600, 16.3)
print('K=', K[0], '+/-', K[1])

#closer zoomed in #inverted fit

ind = find_position(a2elapsed_fs, 59500) 
print(ind)

K = inversefitting(a2_fs[:ind:], a2elapsed_fs[:ind:], a2_lowess[:ind:], 59500/3600, 16.3)
print('K=', K[0], '+/-', K[1])

data/2024_08_07_10_55_58_657286-2-anemometer-microseconds.asc


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

39039
50018


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

K= 0.03292335483884607 +/- 0.00015714186984241258
4962


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.019189403900540226 +/- 2.025848265946597e-05
3752


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

K= 0.01771508357773117 +/- 1.9980514590486994e-05


<a class="anchor" id="7"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Results for Decay Constants $K$</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

As a final result for each Dataset I am taking coeficients from the most zoomed in fits, to maximize the accuracy.

The results are presented in the following table in __[1/m]__:

|           | Dataset 1                | Dataset 2             | Dataset 3             | Dataset 4              |
|-----------|--------------------------|-----------------------|-----------------------|------------------------|
|$K$        | 0.017714232885995263     | 0.009797766410070019  | 0.016536569499209786  | 0.01771508357773117    |
| $\Delta K$| 2.3113446951216764e-05   | 0.00022762001988920643| 2.2482911624194664e-05| 1.9980514590486994e-05 |

One of the firts 3 Datasets should have been taken with not soluable film. I didn't remeber which one, but I can confidently conclude that it is most likely the Dataset 2, as its $K$ is 2 times smaller in comparasment to the other data. The Dataset 4 inclueds data for soluable film. From the results shown abouve we can deduce, that there is no sugnificant (or even any at all) difference in Decay Constants $K$ for "no film" and "soluable film" data (compare Dataset 1 and 4).

One of the goals of this work was to compare my results for $K's$ with the results, measured by M.Bopp in his Bachelor's theasis in 2011. Although we didn't use the same data (only the same instrument, Aeolotron) I am going to site his end results (Table 7.3, which inclueds his experimental and theoretical results) with respect to his data below.

|                            | K [1/m]$\cdot 10^{−4}$   | 
|----------------------------|--------------------------|
|Messung MAVS                | $1. 60 \pm 0. 13$        | 
| Messung Heizdraht Signal-T2| $2. 77 \pm 0. 84$        |
|Messung Heizdraht T2-T1     | $3. 14 \pm 0. 66$        | 
|theor. Modell 2 a           | $1. 665$                 |
|theor. Modell 2 b           | $1. 750$                 | 

From this we see that his results overall are 100 smaller than mine. Any precise explanation or error could not be found. The only reasonable explanation I considered was that in the original code (from Kerstin) the units for water velosity were marked as $"maybe m/s"$. It caused a lot of confusion, as I first considered it to be $cm/s$ to see if that helps my end values to match those of Maximilian, but then the water velocity turned out to be unreasonable and the comparasment with M.Bopp's results didn't actually improve (It is easy to see that $\frac{1}{cm} = 100 \frac{1}{m}$ so my result turned out to be $ \times 10 000$ bigger than his, that is why I switched back to $\frac{1}{m}$ in the end). I think if I considered the velocity to be in $\frac{dm}{s}$ (which would help with the order problem) it wouldn't match the tipical water velocity in the chanel, as it is too quick. In the end I think it might have had something to do with the units, but I can't say what exactly. 

<a class="anchor" id="8"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='black'>Part 2. Friction velocity: qualitative calculation</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

<a class="anchor" id="8.1"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Introduction</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

In this part of the Notebook we will cary out an exemplary evaluation of a dataset to find the friction velosity. It was decided to only evaluate one file, to not overcrowd the notebook, but the same principle will be aplicable to all of the datasets. __To be able to run the following code the very first "import" and "functions" cells as well as the cells from "Part 1: Dataset 1" should already be run. The following code uses the functions defined there.__

<a class="anchor" id="8.2"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Theoretical Basis</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

After an activation of the wind and a certain run-in period, the system reaches a steady-state equilibrium. In this state, the momentum input caused by the wind balances out the momentum loss over the channel. The change in velocity $\frac{\partial u}{\partial t}$ vanishes, and the following force equilibrium is established:

\begin{align}
Aρ_wu∗_w^2 = ρ_w \frac{1}{β^2} \left[A + α_i^2A_i + α_a^2A_a + \frac{1}{2} β^2c_wF \right] \cdot u^2 
\end{align}

From this, a direct relationship between the water velocity $u$ and the wind-induced shear velocity $u∗_w$ can be derived:

\begin{align}
u∗_w = \sqrt{\frac{[A + α_i^2A_i + α_a^2A_a + \frac{1}{2} β^2c_wF]}{A \cdot β^2}} \cdot u = \sqrt{\frac{K \cdot V_w}{A}} \cdot u 
\end{align}

The surface area of the water in the Aeolotron is $ A = (17.927 \pm 0.41) \, \text{m}^2 $, and the water volume is $ V_w = (17.927 \pm 0.45) \, \text{m}^3 $.



<a class="anchor" id="8.3"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Evaluation. Dateset 1</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

The Windspeed log to this Dataset is shown below:

In [16]:
with open("data/20240726-log.txt",'r') as f:
    
    # Read and ignore header lines
    header1 = f.readline()
    header2 = f.readline()
    header3 = f.readline()
    header4 = f.readline()
    
    for line in f:
        print(line)        

10:xx wind an @6.3, skimmen

11:00 Lueftung aus

hw=101.2cm

11:50 Lueftung an wind@8.9

12:05 Lueftung aus

12:55 Lueftung an wind@12.6

13:10 Lueftung aus

13:55 Lueftung an, wind @17.8

hw=100.3

14:10 Lueftung aus

14:55 Lueftung an, wind@25.2

15:10 Lueftung aus

15:50 spindown

16:50 Lueftung an


We are going to use 3 times: 
1) 11:05-11:50 (wind speed = 8.9 Hz) (in seconds: 39900 - 42600 )
2) 12:10 - 12:55 (wind speed = 12.6 Hz) (in seconds: 43800 - 46500)
3) 13:15 - 13:55 (wind speed = 17.8 Hz) (in seconds: 47700 - 50100)

with skipping of the firts 5 mins of data to account for reaching the equilibrium state

In [17]:
#read in the file
a1elapsedlist,a1list, a2elapsedlist, a2list = readthefile("data/2024_07_26_10_38_10_194617-2-anemometer-microseconds-spindown-ab-15.50.asc")

data/2024_07_26_10_38_10_194617-2-anemometer-microseconds-spindown-ab-15.50.asc


In [18]:
#plot the full data

fig, ax = plt.subplots(figsize = (9,6))
#plt.ylim([-0.01,0.2])
#plt.xlim([16.0,16.9])

plt.xlabel('time of day [s]')
plt.ylabel('water velocity [m/s]')
plt.title("Full Data")


ax.plot(a1elapsedlist,a1list,label="water anemometer a1")
ax.plot(a2elapsedlist,a2list,label="water anemometer a2")

plt.grid(visible=True, which='major', color='grey', linestyle='-')
ax.legend()


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
#slice to relevant times

def sli(a2elapsedlist,a2list, start, finish): #start n finish in seconds
    ind1 = find_position(a2elapsedlist, start)
    print(ind1)
    ind2 = find_position(a2elapsedlist, finish)
    print(ind2)
    a2elapsed_s = a1elapsedlist[ind1:ind2:]
    a2_s = a2list[ind1:ind2:]

    fig, ax = plt.subplots(figsize = (9,6))

    plt.xlabel('time of day [s]')
    plt.ylabel('water velocity [m/s]')
    plt.title("Sliced Data")

    ax.plot(a2elapsed_s,a2_s,label="water anemometer a2, 10 mins measurement, start: " + str(start//3600) + ":"+ str((start%3600)//60))

    plt.grid(visible=True, which='major', color='grey', linestyle='-')
    ax.legend(bbox_to_anchor=(0.65, 0.995), loc=1, borderaxespad=0.,fancybox=True, framealpha=1.0,fontsize=10,borderpad=0.2,labelspacing=0.35,handletextpad=0.1)



    plt.show()
    
    return a2elapsed_s, a2_s
    
x1,y1  = sli(a2elapsedlist,a2list, 39900, 42600) #at 11:05
x2,y2  = sli(a2elapsedlist,a2list, 43800, 46500) #at 12:10  
x3,y3  = sli(a2elapsedlist,a2list, 47700, 50100) #at 13:15

115
1994


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

3725
9046


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

12778
21498


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
#filter the noise out

def filter1d(a2elapsed_s, a2_s, start, finish):

    a2elapsed_fs = filter(a2elapsed_s, a2_s)[0]
    a2_fs = filter(a2elapsed_s, a2_s)[1]

    fig, ax = plt.subplots(figsize = (9,6))

    plt.xlabel('time of day [s]')
    plt.ylabel('water velocity [m/s]')
    plt.title("Sliced Filtered Data")

    ax.plot(a2elapsed_fs,a2_fs,label="water anemometer a2, 10 mins measurement, start: " + str(start//3600) + ":"+ str((start%3600)//60))

    plt.grid(visible=True, which='major', color='grey', linestyle='-')
    ax.legend()


    plt.show()
    
    return a2elapsed_fs, a2_fs

x1,y1  = filter1d(x1,y1, 39900, 42600) #at 11:05
x2,y2  = filter1d(x2,y2, 43800, 46500) #at 12:10  
x3,y3  = filter1d(x3,y3, 47700, 50100) #at 13:15

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now we are going to calculate the friction velocities for each of the time periouds according to:

\begin{align}
u∗_w = \sqrt{\frac{[A + α_i^2A_i + α_a^2A_a + \frac{1}{2} β^2c_wF]}{A \cdot β^2}} \cdot u = \sqrt{\frac{K \cdot V_w}{A}} \cdot u 
\end{align}

\begin{align}
\Delta u∗_w =  \sqrt{ \Big(\ \Delta K \frac{1}{2 \sqrt{K} } \sqrt{ \frac{V_w}{A} } \cdot u \Big)\ ^2 + \Big(\ \Delta V_w \frac{1}{2 \sqrt{V_w} } \sqrt{ \frac{K}{A} } \cdot u \Big)\ ^2 + \Big(\ \Delta A \frac{-1}{2 A^{\frac{3}{2}} } \sqrt{ K \cdot V_w } \cdot u \Big)\ ^2  } 
\end{align}

In [21]:
A = 17.927 #m2
V = 17.927 #m3
dA = 0.41 #m2
dV = 0.45 #m3

def frictionvel(K, dK, u):
    u = np.array(u)
    U = np.sqrt(K*V/A)*u
    dU2 = (dK*0.5*(K**(-0.5))*np.sqrt(V/A)*u )**2 + (dV*0.5*(V**(-0.5))*np.sqrt(K/A)*u )**2 + (dA*0.5*(A**(-1.5))*np.sqrt(K*V)*u)**2
    dU = np.sqrt(dU2)
    return (U, dU)

K= 0.017714232885995263
dK = 2.3113446951216764E-05

fig, ax = plt.subplots(figsize = (9,6))

plt.xlabel('time of day [s]')
plt.ylabel('friction velocity [m/s]')
plt.title("Friction Velocity")

ax.errorbar(x1, frictionvel(K, dK, y1)[0], yerr= frictionvel(K, dK, y1)[1], fmt="none", label="water anemometer a2, friction velosity, start: 11:05")
ax.errorbar(x2, frictionvel(K, dK, y2)[0], yerr= frictionvel(K, dK, y2)[1], fmt="none", label="water anemometer a2, friction velosity, start: 12:10")
ax.errorbar(x3, frictionvel(K, dK, y3)[0], yerr= frictionvel(K, dK, y3)[1], fmt="none", label="water anemometer a2, friction velosity, start: 13:15")

plt.grid(visible=True, which='major', color='grey', linestyle='-')
ax.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We assume that the frictin velocity is constant for a given wind velocity. In the following cell we take both the mean and the median, the difference between them is of the order $10^{-5}$ (or even smaller for U_wind = 17.8 Hz) and therefore negligible.

In [22]:
print("the friction velocity velues are all in [m/s]")
print("\n")

c1 = np.median(frictionvel(K, dK, y1)[0])
c2 = np.median(frictionvel(K, dK, y2)[0])
c3 = np.median(frictionvel(K, dK, y3)[0])

print('The median method')
print('The friction velocity for U_wind = 8.9 Hz: {value:.4e}'.format(value=c1))
print('The friction velocity for U_wind = 12.6 Hz: {value:.4e}'.format(value=c2))
print('The friction velocity for U_wind = 17.8 Hz: {value:.4e}'.format(value=c3))
print("\n")

k1 = np.mean(frictionvel(K, dK, y1)[0])
k2 = np.mean(frictionvel(K, dK, y2)[0])
k3 = np.mean(frictionvel(K, dK, y3)[0])

print('The mean method')
print('The friction velocity for U_wind = 8.9 Hz: {value:.4e}'.format(value=k1))
print('The friction velocity for U_wind = 12.6 Hz: {value:.4e}'.format(value=k2))
print('The friction velocity for U_wind = 17.8 Hz: {value:.4e}'.format(value=k3))

the friction velocity velues are all in [m/s]


The median method
The friction velocity for U_wind = 8.9 Hz: 1.5861e-03
The friction velocity for U_wind = 12.6 Hz: 3.6917e-03
The friction velocity for U_wind = 17.8 Hz: 6.7180e-03


The mean method
The friction velocity for U_wind = 8.9 Hz: 1.5187e-03
The friction velocity for U_wind = 12.6 Hz: 3.6608e-03
The friction velocity for U_wind = 17.8 Hz: 6.7129e-03


<a class="anchor" id="8.4"></a>

<div style="display: flex; justify-content: space-between; align-items: center;">
    
    <h1 style="margin: 0; text-align: center; width: 100%">
        <font color='green'>Results for the friction velocity $u*_w$</font>
    </h1>
    <a href="#toc" style="text-decoration: none; font-size: 20px; margin-left: 10px;">&#x21A9;</a>
    
</div>

The reasonable comparasment with the results from Maximilian Bopp is not possible, because the friction velocities must be compared for the same wind velocity. In the table 7.5 M. Bopp presents his friction velocities([cm/s]) for different wind velocities ([m/s]). In the data I evaluated the wind velocity was given in frequancy units [Herz], which is also explained in the M. Bopp's theasis in the section 4.2 . I couldn't find anything that he used to convert it into $m/s$, so I couldn't convert my values to compare. According to Kerstin it is also not possible? Considering that our $K's$ are very different, we wouldn't get comparable results. My results are presented above.